Código do Process Map

In [28]:
import pandas as pd
import pm4py
from graphviz import Digraph

def import_csv(file_path):
    # Carregar o log de eventos a partir do arquivo CSV
    event_log = pd.read_csv(file_path, sep=';')
    
    # Converter a coluna 'timestamp' para o formato de data/hora
    event_log['timestamp'] = pd.to_datetime(event_log['timestamp'])
    
    # Calcular o tempo gasto em cada ação para todos os pacientes
    action_times = {}
    action_freq = {}
    for action in event_log['activity'].unique():
        action_group = event_log[event_log['activity'] == action]
        action_times[action] = action_group['timestamp'].max() - action_group['timestamp'].min()
        action_freq[action] = len(action_group['case_id'].unique())
    
    # Calcular a frequência e o tempo médio de cada situação
    situation_freq = {}
    situation_times = {}
    for i, situation in event_log[['activity', 'case_id']].drop_duplicates().iterrows():
        situation_group = event_log[(event_log['activity'] == situation['activity']) & (event_log['case_id'] == situation['case_id'])]
        situation_freq[situation['activity']] = len(situation_group['case_id'].unique())
        situation_times[situation['activity']] = (situation_group['timestamp'].max() - situation_group['timestamp'].min()) / situation_freq[situation['activity']]
    
    # Calcular a média dos tempos em horas
    total_time_hours = sum(action_times.values(), pd.Timedelta(0)).total_seconds() / 3600
    num_actions = len(action_times)
    if num_actions != 0:
        average_time_hours = total_time_hours / num_actions
    else:
        average_time_hours = 0
    
    # Descobrir o modelo de processo usando o algoritmo Alpha Miner
    event_log = pm4py.format_dataframe(event_log, case_id='case_id', activity_key='activity', timestamp_key='timestamp')
    net, initial_marking, final_marking = pm4py.discover_petri_net_alpha(event_log)
    
    # Criar um objeto Digraph do Graphviz
    dot = Digraph(comment='Process Mining', format='png')
    
    # Adicionar nós ao grafo
    for p in net.places:
        if p.name in situation_times:
            # Adicionar frequência e tempo médio como rótulo do nó
            if situation_times[p.name] >= pd.Timedelta(days=1):
                label = f"{p.name} (Freq.: {situation_freq[p.name]}, Tempo médio: {situation_times[p.name].days:.1f} dias)"
            else:
                label = f"{p.name} (Freq.: {situation_freq[p.name]}, Tempo médio: {situation_times[p.name].seconds / 3600:.1f} horas)"
        else:
            label = p.name
        if p.name == "start":
            dot.node(p.name, label=label, style="filled", fillcolor="green")
        elif p.name == "end":
            dot.node(p.name, label=label, style="filled", fillcolor="green")
        else:
            dot.node(p.name, label=label)

    dot.edge("start", list(net.transitions)[0].name, style="invis")
    dot.edge(list(net.transitions)[-1].name, "end", style="invis")
    
    # Adicionar transições ao grafo
    for t in net.transitions:
        if t.name in action_times:
            label = f"{t.name}\nFreq.: {action_freq[t.name]}\nTempo médio: {action_times[t.name].seconds / 3600:.1f} horas"
        else:
            label = t.name

        dot.node(t.name, label=label, shape='rectangle', style='filled', fillcolor='lightblue')
    
    # Adicionar arestas ao grafo
    for arc in net.arcs:
        dot.edge(arc.source.name, arc.target.name)
       

    # Salvar o gráfico em um arquivo
    dot.render('process_graph', format='png', cleanup=True)
    print("Process graph saved as 'process_graph.png'")

if __name__ == "__main__":
    file_path = "C:\\Caio\\grupo-2\\Bases_de_Dados\\running-example_3.csv"
    import_csv(file_path)

Process graph saved as 'process_graph.png'


Código Rede de Petri

In [ ]:
import pandas as pd
import pm4py
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.visualization.petri_net import visualizer as pn_visualizer
import os

def import_csv(file_path):
    # Carregar o log de eventos a partir do arquivo CSV
    event_log = pd.read_csv(file_path, sep=';')
    
    # Converter a coluna 'timestamp' para o formato de data/hora
    event_log['timestamp'] = pd.to_datetime(event_log['timestamp'])
    
    # Descobrir o modelo de processo usando o algoritmo Alpha Miner
    event_log = pm4py.format_dataframe(event_log, case_id='case_id', activity_key='activity', timestamp_key='timestamp')
    net, initial_marking, final_marking = alpha_miner.apply(event_log)
    
    # Visualizar o modelo de processo
    gviz = pn_visualizer.apply(net, initial_marking, final_marking)
    pn_visualizer.view(gviz)  # Esta linha exibe o gráfico gerado pelo Graphviz

import_csv('C:\\Caio\\grupo-2\\Bases_de_Dados\\running-example_3.csv')

